In [2]:
import pandas as pd
import random
import re
import csv

# Charger le fichier CSV avec pandas
data_path = "./../dataset/communes-departement-region.csv"
villes_df = pd.read_csv(data_path)

# Extraire les noms des communes uniques
villes = villes_df['nom_commune_complet'].dropna().unique().tolist()

# Templates de phrases
templates = [
    "Je veux aller de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Voyage de <VILLE_DEPART> à <VILLE_ARRIVEE> avec un arrêt à <VILLE_ETAPE>.",
    "Je pars de <VILLE_DEPART> pour me rendre à <VILLE_ARRIVEE>.",
    "Trajet entre <VILLE_DEPART> et <VILLE_ARRIVEE>.",
    "Aller de <VILLE_DEPART> à <VILLE_ARRIVEE> via <VILLE_ETAPE>.",
    "Comment se rendre de <VILLE_DEPART> à <VILLE_ARRIVEE> ?",
    "Itinéraire entre <VILLE_DEPART> et <VILLE_ARRIVEE>, en passant par <VILLE_ETAPE>.",
    "Départ de <VILLE_DEPART> pour aller à <VILLE_ARRIVEE>.",
    "Je cherche un itinéraire de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Prendre la route de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Comment aller de <VILLE_DEPART> à <VILLE_ARRIVEE>, avec un passage à <VILLE_ETAPE> ?",
    "Voyager depuis <VILLE_DEPART> jusqu'à <VILLE_ARRIVEE>.",
    "Direction <VILLE_ARRIVEE> depuis <VILLE_DEPART>.",
    "Je voudrais un itinéraire de <VILLE_DEPART> à <VILLE_ARRIVEE> avec une étape à <VILLE_ETAPE>.",
    "Quel est le meilleur chemin entre <VILLE_DEPART> et <VILLE_ARRIVEE> ?",
    "Itinéraire pour aller de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Route de <VILLE_DEPART> vers <VILLE_ARRIVEE>.",
    "Comment rejoindre <VILLE_ARRIVEE> depuis <VILLE_DEPART> ?",
    "Trajet de <VILLE_DEPART> à <VILLE_ARRIVEE>, avec un arrêt à <VILLE_ETAPE>.",
    "Depuis <VILLE_DEPART>, aller à <VILLE_ARRIVEE>.",
    "Je souhaite partir de <VILLE_DEPART> et arriver à <VILLE_ARRIVEE>.",
    "Comment atteindre <VILLE_ARRIVEE> à partir de <VILLE_DEPART> ?",
    "Voyage de <VILLE_DEPART> vers <VILLE_ARRIVEE>, en passant par <VILLE_ETAPE>.",
    "Depuis <VILLE_DEPART>, quel est le meilleur trajet pour <VILLE_ARRIVEE> ?",
    "Aller à <VILLE_ARRIVEE> depuis <VILLE_DEPART>.",
    "Me rendre de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Départ à <VILLE_DEPART> pour arriver à <VILLE_ARRIVEE>.",
    "Itinéraire proposé entre <VILLE_DEPART> et <VILLE_ARRIVEE>.",
    "Comment se rendre de <VILLE_DEPART> à <VILLE_ARRIVEE>, avec une escale à <VILLE_ETAPE> ?",
    "Aller depuis <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Quel trajet pour aller de <VILLE_DEPART> à <VILLE_ARRIVEE> ?",
    "Me rendre à <VILLE_ARRIVEE> depuis <VILLE_DEPART>.",
    "Route à suivre de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "De <VILLE_DEPART>, en route pour <VILLE_ARRIVEE>.",
    "Je veux un itinéraire pour aller de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Quelle est la meilleure route pour aller de <VILLE_DEPART> à <VILLE_ARRIVEE> ?",
    "Je pars de <VILLE_DEPART> et je vais à <VILLE_ARRIVEE>.",
    "Quel est le chemin le plus court entre <VILLE_DEPART> et <VILLE_ARRIVEE> ?",
    "Trajet à effectuer de <VILLE_DEPART> vers <VILLE_ARRIVEE>.",
    "Aller à <VILLE_ARRIVEE> via <VILLE_ETAPE>, en partant de <VILLE_DEPART>.",
    "Depuis <VILLE_DEPART>, je voudrais aller à <VILLE_ARRIVEE>.",
    "Déplacement de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Rejoindre <VILLE_ARRIVEE> en partant de <VILLE_DEPART>.",
    "Je pars de <VILLE_DEPART> pour aller à <VILLE_ARRIVEE>.",
    "Trajet souhaité entre <VILLE_DEPART> et <VILLE_ARRIVEE>.",
    "Comment arriver à <VILLE_ARRIVEE> depuis <VILLE_DEPART> ?",
    "Me rendre à <VILLE_ARRIVEE> avec une étape à <VILLE_ETAPE>, en partant de <VILLE_DEPART>.",
    "Départ prévu à <VILLE_DEPART> pour <VILLE_ARRIVEE>.",
    "Je cherche une route entre <VILLE_DEPART> et <VILLE_ARRIVEE>.",
    "Quel itinéraire pour aller de <VILLE_DEPART> à <VILLE_ARRIVEE> ?",
    "Voyage à partir de <VILLE_DEPART> jusqu'à <VILLE_ARRIVEE>.",
    "Trajet avec une escale à <VILLE_ETAPE> entre <VILLE_DEPART> et <VILLE_ARRIVEE>.",
    "Aller en voiture de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Me diriger vers <VILLE_ARRIVEE> en partant de <VILLE_DEPART>.",
    "Quel itinéraire suivre entre <VILLE_DEPART> et <VILLE_ARRIVEE> ?",
    "Comment rejoindre <VILLE_ARRIVEE> via <VILLE_ETAPE>, en partant de <VILLE_DEPART> ?",
    "Départ de <VILLE_DEPART> pour arriver à <VILLE_ARRIVEE>.",
    "Voyager de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Comment voyager de <VILLE_DEPART> à <VILLE_ARRIVEE> ?",
    "Trajet direct entre <VILLE_DEPART> et <VILLE_ARRIVEE>.",
    "Aller directement de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Quel est l'itinéraire recommandé entre <VILLE_DEPART> et <VILLE_ARRIVEE> ?",
    "Je souhaite aller de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Route conseillée de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "En partant de <VILLE_DEPART>, aller jusqu'à <VILLE_ARRIVEE>.",
    "Voyager via <VILLE_ETAPE> entre <VILLE_DEPART> et <VILLE_ARRIVEE>.",
    "Depuis <VILLE_DEPART>, quelle route prendre pour <VILLE_ARRIVEE> ?",
    "Quelle est la distance entre <VILLE_DEPART> et <VILLE_ARRIVEE> ?",
    "Départ de <VILLE_DEPART> avec un arrêt à <VILLE_ETAPE> avant <VILLE_ARRIVEE>.",
    "Je pars de <VILLE_DEPART> pour rejoindre <VILLE_ARRIVEE>.",
    "En voiture de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Quel est le meilleur trajet de <VILLE_DEPART> à <VILLE_ARRIVEE> ?",
    "Rejoindre <VILLE_ARRIVEE> depuis <VILLE_DEPART>.",
    "Je veux aller à <VILLE_ARRIVEE>, en partant de <VILLE_DEPART>.",
    "Quelle est la meilleure route entre <VILLE_DEPART> et <VILLE_ARRIVEE> ?",
    "Départ de <VILLE_DEPART>, avec un arrêt à <VILLE_ETAPE> avant <VILLE_ARRIVEE>.",
    "Voyager de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Je pars de <VILLE_DEPART> pour aller à <VILLE_ARRIVEE>.",
    "Comment rejoindre <VILLE_ARRIVEE> depuis <VILLE_DEPART> ?",
    "Quel est le trajet idéal entre <VILLE_DEPART> et <VILLE_ARRIVEE> ?",
    "Je souhaite aller de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Route de <VILLE_DEPART> vers <VILLE_ARRIVEE>.",
    "Depuis <VILLE_DEPART>, je veux rejoindre <VILLE_ARRIVEE>.",
    "Me rendre à <VILLE_ARRIVEE> depuis <VILLE_DEPART>.",
    "Voyage entre <VILLE_DEPART> et <VILLE_ARRIVEE>.",
    "Comment aller à <VILLE_ARRIVEE>, en passant par <VILLE_ETAPE>, depuis <VILLE_DEPART> ?",
    "Je cherche un itinéraire entre <VILLE_DEPART> et <VILLE_ARRIVEE>.",
    "Trajet de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Voyage de <VILLE_DEPART> vers <VILLE_ARRIVEE>.",
    "Route de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Je vais de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Quelle est la route pour aller de <VILLE_DEPART> à <VILLE_ARRIVEE> ?",
    "Itinéraire avec un arrêt à <VILLE_ETAPE> entre <VILLE_DEPART> et <VILLE_ARRIVEE>.",
    "Depuis <VILLE_DEPART>, aller directement à <VILLE_ARRIVEE>.",
    "Me rendre de <VILLE_DEPART> à <VILLE_ARRIVEE>.",
    "Voyager de <VILLE_DEPART> à <VILLE_ARRIVEE>, avec une étape à <VILLE_ETAPE>.",
    "Quel est le trajet entre <VILLE_DEPART> et <VILLE_ARRIVEE> ?",
    "Aller de <VILLE_DEPART> vers <VILLE_ARRIVEE>.",
    "Quel est le chemin recommandé entre <VILLE_DEPART> et <VILLE_ARRIVEE> ?",
    "Je pars de <VILLE_DEPART> pour aller à <VILLE_ARRIVEE>, avec un arrêt à <VILLE_ETAPE>."
]


def generate_phrase_and_labels(templates, villes):
    template = random.choice(templates)

    # Sélection de villes de départ, arrivée, et étape
    ville_depart = random.choice(villes)
    ville_arrivee = random.choice(villes)
    while ville_arrivee == ville_depart:
        ville_arrivee = random.choice(villes)

    ville_etape = None
    if "<VILLE_ETAPE>" in template:
        ville_etape = random.choice(villes)
        while ville_etape == ville_depart or ville_etape == ville_arrivee:
            ville_etape = random.choice(villes)

    # Création des tokens et des labels
    tokens = []
    labels = []

    # Utilisation de regex pour détecter et remplacer les balises
    def replace_and_label(match, ville, label_prefix):
        ville_tokens = ville.split()
        tokens.extend(ville_tokens)
        labels.extend([f"B-{label_prefix}"] + [f"I-{label_prefix}"] * (len(ville_tokens) - 1))

    # Remplacement des balises par les noms de villes et attribution des labels
    pos = 0
    for match in re.finditer(r"<(VILLE_DEPART|VILLE_ARRIVEE|VILLE_ETAPE)>", template):
        # Ajouter les mots entre les balises en tant que tokens normaux avec le label 'O'
        if match.start() > pos:
            words_between = template[pos:match.start()].split()
            tokens.extend(words_between)
            labels.extend(["O"] * len(words_between))

        # Identification de la balise et application du remplacement
        if match.group(0) == "<VILLE_DEPART>":
            replace_and_label(match, ville_depart, "depart")
        elif match.group(0) == "<VILLE_ARRIVEE>":
            replace_and_label(match, ville_arrivee, "arrive")
        elif match.group(0) == "<VILLE_ETAPE>" and ville_etape:
            replace_and_label(match, ville_etape, "etape")

        pos = match.end()

    # Ajouter le reste de la phrase après la dernière balise
    if pos < len(template):
        remaining_words = template[pos:].split()
        tokens.extend(remaining_words)
        labels.extend(["O"] * len(remaining_words))

    return {"tokens": tokens, "labels": labels}

# Génération de 10,000 exemples labellisés
dataset_size = 10000
dataset = []

for _ in range(dataset_size):
    labelled_data = generate_phrase_and_labels(templates, villes)
    dataset.append(labelled_data)

# Sauvegarder le dataset dans un fichier CSV
output_file = "dataset_phrases_labelled.csv"
with open(output_file, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["tokens", "labels"])  # Noms des colonnes
    for item in dataset:
        writer.writerow([item['tokens'], item['labels']])

print(f"Dataset de {dataset_size} phrases labellisées généré et sauvegardé dans {output_file}.")

Dataset de 10000 phrases labellisées généré et sauvegardé dans dataset_phrases_labelled.csv.
